In [1]:
# Warning: this notebook doesn't work on Apple M1.

In [2]:
using Pkg
Pkg.instantiate()
using IJulia
using LinearAlgebra
using ADIOS2
using GR
using Plots

In [3]:
adios = ADIOS2.adios_init_serial(joinpath(pwd(),"adios2.xml"))     # Use the configurations defined in "adios2.xml"...
io = ADIOS2.declare_io(adios, "readerIO")                          # ... in the section "readerIO"
engine = ADIOS2.open(io, "diffusion2D.bp", mode_read)              # Open the file/stream "diffusion2D.bp"
engine_type(io)                                                    # Check the type of the engine: BP4/SST...

"BP4"

In [4]:
inquire_all_variables(io)     # There are no available variables before the first step when using SST

1-element Vector{Variable}:
 Variable(temperature)

In [5]:
nprocessed=0
while begin_step(engine, step_mode_read, 100.0) != step_status_end_of_stream   # Begin the next ADIOS2 read step while there is any
    T_id = inquire_variable(io, "temperature")                                 # Get a handle for the variable T. ATTENTION: T_id is only valid for the current step when using streaming (SST or file streaming); thus, moving it into the if statement below would lead to an error!
    if nprocessed == 0
        nxy_global = shape(T_id)                                               # Extract meta data
        nxy        = count(T_id)                                               # ...
        T_type     = type(T_id)                                                # ...
        global T = zeros(T_type, nxy)                                          # Preallocate memory for T using the meta data
        print(nxy_global, nxy, T_type); sleep(4)                               # Note that nxy==nxy_global as we read only with one process
    end  
    get(engine, T_id, T)                                                       # Add T to variable(s) for reading (SetSelection() would be needed in addition if we were reading the data with multiple MPI processes: this would make sure that every process reads "his" block)
    end_step(engine)                                                           # End ADIOS2 read step (includes normally the actual reading of data) 
    IJulia.clear_output(true)                                                  # Pass wait=true to wait until new ouput before clearing; this prevents flickering
    plot_title = "Temperature at step " * string(nprocessed)                   # Plot the temperature
    display(Plots.contourf(T, title=plot_title, size=(1700,1700), 
                           xtickfontsize=30, ytickfontsize=30,                 # ...
                           xguidefontsize=30, yguidefontsize=30, 
                           legendfontsize=30, titlefontsize=40))               # ...
    
    nprocessed += 1
end

(0x000000000000007e, 0x000000000000007e)(0x000000000000007e, 0x000000000000007e)Float64

LoadError: MethodError: no method matching Int64(::Base.RefValue{UInt64})

[0mClosest candidates are:
[0m  (::Type{T})([91m::AbstractChar[39m) where T<:Union{Int32, Int64}
[0m[90m   @[39m [90mBase[39m [90m[4mchar.jl:51[24m[39m
[0m  (::Type{T})([91m::AbstractChar[39m) where T<:Union{AbstractChar, Number}
[0m[90m   @[39m [90mBase[39m [90m[4mchar.jl:50[24m[39m
[0m  (::Type{T})([91m::BigInt[39m) where T<:Union{Int128, Int16, Int32, Int64, Int8}
[0m[90m   @[39m [90mBase[39m [90m[4mgmp.jl:367[24m[39m
[0m  ...


In [6]:
close(engine)

error_none::Error = 0